# 小红书爬虫 - 关键字搜索与封面下载

**功能说明:**
- 根据关键字搜索小红书笔记
- 提取笔记标题、链接、封面图片
- 批量下载封面thumbnail
- 导出数据为CSV格式

**使用方法:**
1. 安装依赖: `pip install requests pandas fake-useragent pillow`
2. 设置搜索关键字
3. 运行爬虫获取数据
4. 下载封面图片到本地

**注意:** 仅供学习研究使用，请遵守网站服务条款

In [ ]:
# 导入必要的库
import requests
import json
import time
import os
import pandas as pd
from pathlib import Path
from typing import List, Dict
from datetime import datetime

# 图片处理
from PIL import Image
from io import BytesIO

# User-Agent伪装
try:
    from fake_useragent import UserAgent
    ua = UserAgent()
except ImportError:
    print("Warning: fake_useragent not installed, using default UA")
    ua = None

print("✓ 库导入成功")

In [ ]:
# 配置参数
class CrawlerConfig:
    """爬虫配置"""
    # 搜索关键字
    KEYWORD = "美食"  # 修改为你想搜索的关键字
    
    # 获取数量
    MAX_ITEMS = 20  # 最多获取多少条笔记
    
    # 请求延迟（秒）
    REQUEST_DELAY = 1  # 每次请求间隔，避免频繁访问
    
    # 保存路径
    OUTPUT_DIR = "xiaohongshu_data"  # 数据保存目录
    IMAGE_DIR = "xiaohongshu_images"  # 图片保存目录
    
    # 请求头
    HEADERS = {
        'User-Agent': ua.random if ua else 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Referer': 'https://www.xiaohongshu.com/',
    }

# 创建输出目录
os.makedirs(CrawlerConfig.OUTPUT_DIR, exist_ok=True)
os.makedirs(CrawlerConfig.IMAGE_DIR, exist_ok=True)

print(f"配置完成:")
print(f"  关键字: {CrawlerConfig.KEYWORD}")
print(f"  数量: {CrawlerConfig.MAX_ITEMS}")
print(f"  保存路径: {CrawlerConfig.OUTPUT_DIR}/")

In [ ]:
class XiaohongshuCrawler:
    """小红书爬虫类"""
    
    def __init__(self, config: CrawlerConfig):
        self.config = config
        self.session = requests.Session()
        self.session.headers.update(config.HEADERS)
        self.results = []
    
    def search_notes(self, keyword: str, max_items: int = 20) -> List[Dict]:
        """
        搜索小红书笔记
        
        注意: 小红书有严格的反爬虫机制，直接API请求可能失败
        这里提供两种方案:
        1. 模拟API请求（可能需要登录cookie）
        2. 使用selenium/playwright模拟浏览器（更稳定但较慢）
        """
        print(f"开始搜索关键字: {keyword}")
        
        # 方案1: 尝试直接API请求 (可能需要更新API地址和参数)
        # 小红书的搜索API经常变化，这里提供基础模板
        search_url = "https://edith.xiaohongshu.com/api/sns/web/v1/search/notes"
        
        params = {
            'keyword': keyword,
            'page': 1,
            'page_size': max_items,
            'search_id': int(time.time() * 1000),
        }
        
        try:
            response = self.session.get(search_url, params=params, timeout=10)
            print(f"状态码: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                return self._parse_search_results(data)
            else:
                print(f"请求失败: {response.status_code}")
                print("建议: 使用selenium方案或添加登录cookie")
                return self._demo_data()  # 返回演示数据
                
        except Exception as e:
            print(f"请求异常: {e}")
            print("返回演示数据用于测试...")
            return self._demo_data()
    
    def _parse_search_results(self, data: dict) -> List[Dict]:
        """解析搜索结果"""
        results = []
        
        try:
            items = data.get('data', {}).get('items', [])
            
            for item in items[:self.config.MAX_ITEMS]:
                note = item.get('note_card', {})
                
                result = {
                    'note_id': note.get('note_id', ''),
                    'title': note.get('display_title', ''),
                    'desc': note.get('desc', ''),
                    'type': note.get('type', ''),
                    'cover_url': note.get('cover', {}).get('url', ''),
                    'user_name': note.get('user', {}).get('nickname', ''),
                    'user_id': note.get('user', {}).get('user_id', ''),
                    'liked_count': note.get('interact_info', {}).get('liked_count', 0),
                    'note_url': f"https://www.xiaohongshu.com/explore/{note.get('note_id', '')}",
                }
                results.append(result)
            
            print(f"成功解析 {len(results)} 条笔记")
            
        except Exception as e:
            print(f"解析失败: {e}")
        
        return results
    
    def _demo_data(self) -> List[Dict]:
        """返回演示数据（用于测试）"""
        print("⚠️  使用演示数据（API请求失败）")
        return [
            {
                'note_id': 'demo001',
                'title': '演示笔记1 - 美食分享',
                'desc': '这是一个演示数据，实际使用需要配置cookie或使用selenium',
                'type': 'normal',
                'cover_url': 'https://picsum.photos/400/600?random=1',
                'user_name': '演示用户1',
                'user_id': 'demo_user_1',
                'liked_count': 1234,
                'note_url': 'https://www.xiaohongshu.com/explore/demo001',
            },
            {
                'note_id': 'demo002',
                'title': '演示笔记2 - 旅行攻略',
                'desc': '这是一个演示数据，实际使用需要配置cookie或使用selenium',
                'type': 'video',
                'cover_url': 'https://picsum.photos/400/600?random=2',
                'user_name': '演示用户2',
                'user_id': 'demo_user_2',
                'liked_count': 5678,
                'note_url': 'https://www.xiaohongshu.com/explore/demo002',
            },
        ]
    
    def download_image(self, url: str, save_path: str) -> bool:
        """下载图片"""
        try:
            response = self.session.get(url, timeout=10)
            if response.status_code == 200:
                # 保存图片
                with open(save_path, 'wb') as f:
                    f.write(response.content)
                return True
            else:
                print(f"下载失败: {url} (状态码: {response.status_code})")
                return False
        except Exception as e:
            print(f"下载异常: {url} ({e})")
            return False
    
    def download_thumbnails(self, results: List[Dict]) -> int:
        """批量下载封面"""
        print(f"\n开始下载封面图片...")
        success_count = 0
        
        for i, item in enumerate(results, 1):
            cover_url = item.get('cover_url', '')
            if not cover_url:
                print(f"[{i}/{len(results)}] 跳过: 无封面URL")
                continue
            
            # 生成文件名
            note_id = item.get('note_id', f'unknown_{i}')
            ext = '.jpg'  # 默认jpg，也可以从URL解析
            save_path = os.path.join(self.config.IMAGE_DIR, f"{note_id}{ext}")
            
            # 如果已存在则跳过
            if os.path.exists(save_path):
                print(f"[{i}/{len(results)}] 跳过: {note_id} (已存在)")
                success_count += 1
                continue
            
            # 下载
            print(f"[{i}/{len(results)}] 下载: {note_id}...", end=' ')
            if self.download_image(cover_url, save_path):
                print("✓")
                success_count += 1
                item['local_image_path'] = save_path
            else:
                print("✗")
            
            # 延迟
            time.sleep(self.config.REQUEST_DELAY)
        
        print(f"\n下载完成: {success_count}/{len(results)}")
        return success_count
    
    def save_to_csv(self, results: List[Dict], filename: str = None) -> str:
        """保存为CSV"""
        if not filename:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"xiaohongshu_{self.config.KEYWORD}_{timestamp}.csv"
        
        filepath = os.path.join(self.config.OUTPUT_DIR, filename)
        
        df = pd.DataFrame(results)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        
        print(f"\n数据已保存: {filepath}")
        return filepath

print("✓ 爬虫类定义完成")

In [ ]:
# 运行爬虫
crawler = XiaohongshuCrawler(CrawlerConfig)

# 搜索笔记
results = crawler.search_notes(
    keyword=CrawlerConfig.KEYWORD,
    max_items=CrawlerConfig.MAX_ITEMS
)

# 显示结果预览
if results:
    print(f"\n{'='*60}")
    print(f"搜索结果预览 (共 {len(results)} 条):")
    print(f"{'='*60}\n")
    
    for i, item in enumerate(results[:5], 1):  # 只显示前5条
        print(f"{i}. {item['title']}")
        print(f"   作者: {item['user_name']}")
        print(f"   点赞: {item['liked_count']}")
        print(f"   链接: {item['note_url']}")
        print(f"   封面: {item['cover_url'][:60]}..." if len(item['cover_url']) > 60 else f"   封面: {item['cover_url']}")
        print()
    
    if len(results) > 5:
        print(f"... 还有 {len(results) - 5} 条结果\n")
else:
    print("⚠️  未获取到数据")

In [ ]:
# 下载封面图片
if results:
    success = crawler.download_thumbnails(results)
    print(f"\n图片保存目录: {CrawlerConfig.IMAGE_DIR}/")
else:
    print("没有可下载的图片")

In [ ]:
# 保存数据为CSV
if results:
    csv_path = crawler.save_to_csv(results)
    
    # 显示DataFrame
    df = pd.DataFrame(results)
    print("\nDataFrame预览:")
    display(df[['title', 'user_name', 'liked_count', 'type']].head(10))
else:
    print("没有可保存的数据")

---

## 进阶方案: 使用Selenium/Playwright

如果上述API请求失败，可以使用浏览器自动化工具:

```python
# 安装: pip install playwright
# 初始化: playwright install

from playwright.sync_api import sync_playwright

def search_with_browser(keyword: str):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        
        # 访问搜索页面
        page.goto(f'https://www.xiaohongshu.com/search_result?keyword={keyword}')
        page.wait_for_timeout(3000)
        
        # 提取数据
        notes = page.query_selector_all('.note-item')  # 需要根据实际HTML调整
        
        results = []
        for note in notes:
            title = note.query_selector('.title').inner_text()
            cover = note.query_selector('img').get_attribute('src')
            results.append({'title': title, 'cover_url': cover})
        
        browser.close()
        return results
```

---

## 使用说明

### 1. 安装依赖
```bash
pip install requests pandas fake-useragent pillow
# 可选: pip install playwright selenium
```

### 2. 修改配置
在 `config` cell中修改:
- `KEYWORD`: 搜索关键字
- `MAX_ITEMS`: 获取数量
- `REQUEST_DELAY`: 请求延迟

### 3. 运行顺序
1. 导入库 → 2. 配置 → 3. 定义爬虫类 → 4. 运行搜索 → 5. 下载图片 → 6. 保存数据

### 4. 反爬虫处理
- **添加Cookie**: 在浏览器登录后复制cookie到 `HEADERS`
- **使用代理**: 配置代理IP池
- **降低频率**: 增加 `REQUEST_DELAY`
- **浏览器方案**: 使用playwright/selenium

### 5. 输出文件
- CSV数据: `xiaohongshu_data/`
- 封面图片: `xiaohongshu_images/`

---

**免责声明**: 本代码仅供学习研究使用，请遵守小红书服务条款和robots.txt规则，不得用于商业用途或恶意爬取。